In [10]:
def load_dataset(file_name):
    import pandas as pd
    import json
    
    df = []
    with open('/mnt/ceph/storage/data-in-progress/data-research/web-search/ECIR-22/ecir22-clickbait-spoiling/' + file_name) as f:
        for i in f:
            i = json.loads(i)
            tweet = i['postText']
            article_title = i['targetTitle']
            article = ' '.join(i['targetParagraphs'])
            label = i['tags']
            
            assert len(tweet) == 1
            tweet = tweet[0]
            
            assert len(label) == 1
            label = label[0]
            
            if label not in ['phrase', 'phrases', 'passage', 'multi']:
                print(label)
                
            assert label in ['phrase', 'phrases', 'passage', 'multi']
            
            if label == 'multi':
                continue
            
            df += [{'text': tweet + ' - ' + article_title + article, 
                    'labels': (label == 'phrase' or label == 'phrases')}]

    return pd.DataFrame(df)   
            
    
test_dataset = load_dataset('test.jsonl')
train_dataset = load_dataset('train.jsonl')
validation_dataset = load_dataset('validation.jsonl')

In [2]:
configurations = []

for learn_rate in [4e-5]:
    for warumup_ratio in [0.02, 0.06]:
        for batch_size in [8, 16]:
            for seq_length in [256, 384, 512]:
                configurations += [{
                    "overwrite_output_dir": True,
                    "num_train_epochs": 10,
                    "fp16": False,
                    "train_batch_size": batch_size,
                    "gradient_accumulation_steps": 4,
                    "evaluate_during_training": True,
                    "max_seq_length": seq_length,
                    "learning_rate": learn_rate,
                    "early_stopping_consider_epochs": True,
                    "early_stopping_delta": 0.01,
                    "early_stopping_metric": "acc",
                    "early_stopping_metric_minimize": False,
                    "early_stopping_patience": 3,
                    "evaluate_during_training_steps": 331,
                    "output_dir": "outputs/",
                    'warmup_ratio': warumup_ratio,
                    # 'warmup_steps': 0, # usually empfohlen, warmup_ratio ist prozentuales äquivalent 
                    #                    # <> überschreiben sich gegenseitig
                    'save_steps': 2000,
                    "manual_seed": 12345
                }]

In [10]:
def train_model(config, num):
    from simpletransformers.classification import ClassificationModel
    import sklearn
    
    config["output_dir"] = "outputs/roberta_concat_" + str(num)
    
    model = ClassificationModel("roberta", "roberta-large", args = config)
    # model.train_model(train_dataset)
    model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

In [ ]:
for config, num in zip(configurations, range(len(configurations))):
    print('Running configuration number', num)
    
    train_model(config, num)

Running configuration number 0


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

  0%|          | 0/2641 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/331 [00:00<?, ?it/s]

...

  0%|          | 0/657 [00:00<?, ?it/s]

Running Epoch 3 of 10:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

Running Epoch 4 of 10:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

In [4]:
from simpletransformers.classification import ClassificationModel

model = ClassificationModel("roberta", "roberta-large", args = args)
# model.train_model(train_dataset)
model.train_model(train_dataset, eval_df=validation_dataset, acc=sklearn.metrics.accuracy_score)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'clas

  0%|          | 0/2641 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

...

  0%|          | 0/657 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/331 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

  0%|          | 0/657 [00:00<?, ?it/s]

(410,
 defaultdict(list,
             {'global_step': [82, 164, 246, 328, 331, 410],
              'train_loss': [0.618156909942627,
               0.23921604454517365,
               0.3293249309062958,
               0.024526961147785187,
               0.05657017230987549,
               0.005580205004662275],
              'mcc': [0.5328399399577675,
               0.5335435891935089,
               0.5747947439246016,
               0.5959498221517093,
               0.5939630284450473,
               0.5736127263299481],
              'tp': [226, 277, 278, 251, 242, 266],
              'tn': [274, 226, 239, 272, 279, 251],
              'fp': [48, 96, 83, 50, 43, 71],
              'fn': [109, 58, 57, 84, 93, 69],
              'auroc': [0.8248539909149902,
               0.8608788356354872,
               0.8597849263001761,
               0.8600908500973394,
               0.8599332529897099,
               0.8573189950866784],
              'auprc': [0.828902047498034,
       

In [4]:
from glob import glob
from simpletransformers.classification import ClassificationModel
import sklearn
import pandas as pd
from tqdm import tqdm

df = []

for config, num in zip(configurations, range(len(configurations))):
    directory = "outputs/roberta_concat_" + str(num) + "/"
    
    for checkpoint in tqdm(glob(directory + "checkpoint-*")):
        print(checkpoint)
        model = ClassificationModel("roberta", checkpoint)
        
        valid_acc = model.eval_model(validation_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        test_acc = model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)[0]['acc']
        
        df += [{"checkpoint": checkpoint, "valid_acc": valid_acc, "test_acc": test_acc,
               "config": config}]
        
df = pd.DataFrame(df)
df

  0%|          | 0/22 [00:00<?, ?it/s]

outputs/roberta_concat_0/checkpoint-246-epoch-3


  0%|          | 0/657 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/826 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/83 [00:00<?, ?it/s]

  0%|          | 0/826 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

100%|██████████| 11/11 [08:56<00:00, 48.78s/it]


,checkpoint,valid_acc,test_acc,config
0,outputs/roberta_concat_0/checkpoint-246-epoch-3,0.797565,0.782082,"{'overwrite_output_dir': True, 'num_train_epoc..."
1,outputs/roberta_concat_0/checkpoint-123-epoch-3,0.779300,0.786925,"{'overwrite_output_dir': True, 'num_train_epoc..."
2,outputs/roberta_concat_0/checkpoint-656-epoch-8,0.768645,0.786925,"{'overwrite_output_dir': True, 'num_train_epoc..."
3,outputs/roberta_concat_0/checkpoint-820-epoch-10,0.767123,0.790557,"{'overwrite_output_dir': True, 'num_train_epoc..."
4,outputs/roberta_concat_0/checkpoint-410-epoch-5,0.757991,0.792978,"{'overwrite_output_dir': True, 'num_train_epoc..."
...,...,...,...,...
143,outputs/roberta_concat_11/checkpoint-331,0.792998,0.794189,"{'overwrite_output_dir': True, 'num_train_epoc..."
144,outputs/roberta_concat_11/checkpoint-164-epoch-4,0.788432,0.795400,"{'overwrite_output_dir': True, 'num_train_epoc..."
145,outputs/roberta_concat_11/checkpoint-369-epoch-9,0.786910,0.791768,"{'overwrite_output_dir': True, 'num_train_epoc..."
146,outputs/roberta_concat_11/checkpoint-205-epoch-5,0.783866,0.794189,"{'overwrite_output_dir': True, 'num_train_epoc..."


In [5]:
df.to_json('outputs/roberta_concat_eval_results.json')

In [7]:
import pandas as pd

df = pd.read_json('outputs/roberta_concat_eval_results.json')

In [8]:
df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False)

,checkpoint,valid_acc,test_acc,config
135,outputs/roberta_concat_10/checkpoint-205-epoch-5,0.806697,0.803874,"{'overwrite_output_dir': True, 'num_train_epoc..."
86,outputs/roberta_concat_6/checkpoint-328-epoch-4,0.803653,0.791768,"{'overwrite_output_dir': True, 'num_train_epoc..."
110,outputs/roberta_concat_8/checkpoint-328-epoch-4,0.802131,0.780872,"{'overwrite_output_dir': True, 'num_train_epoc..."
137,outputs/roberta_concat_11/checkpoint-123-epoch-3,0.802131,0.808717,"{'overwrite_output_dir': True, 'num_train_epoc..."
118,outputs/roberta_concat_9/checkpoint-82-epoch-2,0.800785,0.788136,"{'overwrite_output_dir': True, 'num_train_epoc..."
...,...,...,...,...
128,outputs/roberta_concat_10/checkpoint-41-epoch-1,0.509893,0.512107,"{'overwrite_output_dir': True, 'num_train_epoc..."
35,outputs/roberta_concat_2/checkpoint-656-epoch-8,0.509893,0.512107,"{'overwrite_output_dir': True, 'num_train_epoc..."
34,outputs/roberta_concat_2/checkpoint-246-epoch-3,0.509893,0.512107,"{'overwrite_output_dir': True, 'num_train_epoc..."
37,outputs/roberta_concat_2/checkpoint-410-epoch-5,0.490107,0.487893,"{'overwrite_output_dir': True, 'num_train_epoc..."


In [11]:
# Based on the validation data above, we select outputs/roberta_concat_10/checkpoint-205-epoch-5
from simpletransformers.classification import ClassificationModel
import sklearn

model = ClassificationModel("roberta", "outputs/roberta_concat_10/checkpoint-205-epoch-5")
model.eval_model(test_dataset, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/826 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

({'mcc': 0.6086913163097103,
  'tp': 332,
  'tn': 332,
  'fp': 71,
  'fn': 91,
  'auroc': 0.8747396887410614,
  'auprc': 0.8729508399832651,
  'acc': 0.8038740920096852,
  'eval_loss': 0.4974653111824479},
 array([[-2.01746392,  2.98581624],
        [-2.41791868,  3.32718134],
        [ 0.73960215, -1.00264728],
        ...,
        [-1.70847011,  2.99427009],
        [-1.51753819,  2.9932549 ],
        [-2.02497983,  2.7740624 ]]),
 [])

In [9]:
for _, i in [i for i in df[df['checkpoint'].str.contains('epoch')].sort_values('valid_acc', ascending=False).iterrows()][:5]:
    print(i['valid_acc'], i['config'])

0.8066971081000001 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 16, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 384, 'learning_rate': 4e-05, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs/', 'warmup_ratio': 0.06, 'save_steps': 2000, 'manual_seed': 12345}
0.8036529680000001 {'overwrite_output_dir': True, 'num_train_epochs': 10, 'fp16': False, 'train_batch_size': 8, 'gradient_accumulation_steps': 4, 'evaluate_during_training': True, 'max_seq_length': 256, 'learning_rate': 4e-05, 'early_stopping_consider_epochs': True, 'early_stopping_delta': 0.01, 'early_stopping_metric': 'acc', 'early_stopping_metric_minimize': False, 'early_stopping_patience': 3, 'evaluate_during_training_steps': 331, 'output_dir': 'outputs/', 'warm